In [ ]:
#First Install
!pip install torchmetrics --quiet  --user
!pip install pytorch_lightning --quiet  --user
!pip install install lightning-bolts --quiet  --user


In [ ]:
import os
import sys

In [ ]:
# ! pip install --user --quiet "torchmetrics>=0.7, <0.12" "setuptools==67.4.0" "ipython[notebook]>=8.0.0, <8.12.0" "torch>=1.8.1, <1.14.0" "torchvision" "pytorch-lightning>=1.4, <2.0.0"


!pip install pytorch-lightning==1.9.0 --user --quiet
!pip install git+https://github.com/PytorchLightning/lightning-bolts.git@master --upgrade
#!pip install pytorch-lightning==2.0.0 --user --quiet
!pip install lightning-bolts==0.6.0.post1 --user --quiet


!pip install lightning-bolts --user --quiet

In [ ]:
import torchmetrics as tm

In [ ]:
import os
import tempfile
import time

import mlflow
from ray import air, tune
from ray.air import session
#from ray.air.integrations.mlflow import MLflowLoggerCallback, setup_mlflow
from mlflow_ray_utils.mlflow_callback import *

In [ ]:
def evaluation_fn(step, width, height):
    return (0.1 + width * step / 100) ** (-1) + height * 0.1


def train_function(config):
    width, height = config["width"], config["height"]

    for step in range(config.get("steps", 100)):
        # Iterative training function - can be any arbitrary training procedure
        intermediate_score = evaluation_fn(step, width, height)
        # Feed the score back to Tune.
        session.report({"iterations": step, "mean_loss": intermediate_score})
        time.sleep(0.1)

In [ ]:
def tune_with_callback(mlflow_tracking_uri, experiment_name,finish_fast=False):
    cb = MyMLflowLoggerCallback(
                    tracking_uri=mlflow_tracking_uri,
                    experiment_name=experiment_name,
                    save_artifact=True,
                )
    tuner = tune.Tuner(
        train_function,
        tune_config=tune.TuneConfig(num_samples=5),        
        run_config=air.RunConfig(
            name="mlflow",
            callbacks=[
                cb
            ],
        ),
        param_space={
            "width": tune.randint(10, 100),
            "height": tune.randint(0, 100),
            "steps": 5 if finish_fast else 100,
        },
    )
    results = tuner.fit()
    cb.log_end_parent_run()

In [ ]:
import mlflow
import os
import ray
import logging
from mlflow_ray_utils.mlflow_callback import *

temp_dir='/tmp'

if not ray.is_initialized():
    service_host = os.environ["RAY_HEAD_SERVICE_HOST"]
    service_port = os.environ["RAY_HEAD_SERVICE_PORT"]
    address=f"ray://{service_host}:{service_port}"
    #temp_dir='/mnt/data//{}/'.format(os.environ['DOMINO_PROJECT_NAME']) #set to a dataset
    ray.init(address=address, _temp_dir=temp_dir,runtime_env={"py_modules": ['mlflow_ray_utils']})

print('Ray Initializied')
print(f'Ray Host={service_host} and Ray Port={service_port}')

experiment_name = 'RAYv7'+'-' + os.environ['DOMINO_STARTING_USERNAME'] + '-' + os.environ['DOMINO_PROJECT_NAME']
tune_with_callback(os.environ['CLUSTER_MLFLOW_TRACKING_URI'], experiment_name,finish_fast=True)

#tune_with_setup(os.environ['CLUSTER_MLFLOW_TRACKING_URI'], experiment_name,finish_fast=True)
df = mlflow.search_runs(
        [mlflow.get_experiment_by_name(experiment_name).experiment_id]
    )
print(df)
